Import Libraries and variables

In [41]:
import pandas as pd

df=pd.read_csv("./datasets/all_vehicles.csv")

noise_words=["brand new","b/n","brand n","brandnew","new","used","1st","1st owner","petrol","diesel","hybrid","electric","Anniversary",
"second","2nd owner","company maintained","first","owner","owners","manual","auto","automatic","m/t","a/t","mt","at",
"full option","full options","full-option","full-options","full optioned","full-option","highest spec","high spec","full spec","Pre Order","Moon Roof",
"moonroof","(brandnew)","(brand new)","(moonroof)"]

df.head()

,Manufacturer,Model,Year,Fuel Type,Price,Mileage,Average Price (0-20k km),Average Price (20k-50k km),Average Price (50k-100k km),Average Price (100k+ km),Condition
0,Audi,Q7 Platinum Edition,2025.0,Petrol,73000000,0,73000000.0,NaN,NaN,NaN,NaN
1,Audi,Q7 Quattro,2025.0,Petrol,75000000,50,75000000.0,NaN,35825000.0,25250000.0,NaN
2,Audi,A1 TFSI 25,2024.0,Petrol,15450000,3350,15650000.0,NaN,NaN,NaN,NaN
3,Audi,A3,2024.0,Petrol,24500000,7000,24000000.0,NaN,13500000.0,NaN,NaN
4,Audi,A3 S Line Highest Spec,2024.0,Petrol,25000000,300,25000000.0,NaN,NaN,NaN,NaN


In [42]:
#adding seating capacity column
if "Seating Capacity" not in df.columns:
    col_index=df.columns.get_loc("Fuel Type")+1
    df.insert(col_index,"Seating Capacity",None)
else:
    print("Seating Capacity column already exists")

Normalization

In [44]:
#Lowercase and remove white spaces from the end and beginning
df["Model"]=df["Model"].str.lower().str.strip()

#Shorting
df=df.sort_values(by=["Manufacturer","Model","Year"],ascending=[True,True,False])

#remove noise words
for word in noise_words:
    df["Model"]=df["Model"].str.replace(word,"",regex=True)
df["Model"]=df["Model"].str.replace(r"\s+"," ",regex=True).str.strip()

df.head()

,Manufacturer,Model,Year,Fuel Type,Seating Capacity,Price,Mileage,Average Price (0-20k km),Average Price (20k-50k km),Average Price (50k-100k km),Average Price (100k+ km),Condition
45,Audi,a1,2018.0,Petrol,None,9600000,72000,NaN,NaN,9200000.0,NaN,NaN
66,Audi,a1,2017.0,Petrol,None,8800000,91600,NaN,NaN,9200000.0,NaN,NaN
75,Audi,a1,2016.0,Petrol,None,9350000,87000,NaN,NaN,9350000.0,NaN,NaN
46,Audi,a1 s-line rs1,2018.0,Petrol,None,10890000,27000,NaN,10890000.0,NaN,NaN,NaN
76,Audi,a1 tfsi,2016.0,Petrol,None,9700000,43000,NaN,9700000.0,NaN,NaN,NaN


In [50]:
##Seatsing Capacity 
audi_series={"a1":5,"a3":5,"a4":5,"a5":5,"a6":5,"a7":5,"a8":5,
             "q2":5,"q3":5,"q5":5,"q7":7,"q8":5,"e-tron":5}
for audi_model,seats in audi_series.items():
    df.loc[(df["Manufacturer"]=="Audi")&(df["Model"].str.contains(audi_model)),"Seating Capacity"]=seats

df.head()
print(f"Added seating capacity for {df['Seating Capacity'].notnull().sum()} Audi models")


Added seating capacity for 94 Audi models


Dropping 

In [48]:
df=df.drop_duplicates(subset=["Manufacturer","Model","Year"],keep="first")

df.head()

,Manufacturer,Model,Year,Fuel Type,Seating Capacity,Price,Mileage,Average Price (0-20k km),Average Price (20k-50k km),Average Price (50k-100k km),Average Price (100k+ km),Condition
45,Audi,a1,2018.0,Petrol,5,9600000,72000,NaN,NaN,9200000.0,NaN,NaN
66,Audi,a1,2017.0,Petrol,5,8800000,91600,NaN,NaN,9200000.0,NaN,NaN
75,Audi,a1,2016.0,Petrol,5,9350000,87000,NaN,NaN,9350000.0,NaN,NaN
46,Audi,a1 s-line rs1,2018.0,Petrol,5,10890000,27000,NaN,10890000.0,NaN,NaN,NaN
76,Audi,a1 tfsi,2016.0,Petrol,5,9700000,43000,NaN,9700000.0,NaN,NaN,NaN


In [49]:
df.to_csv("./datasets/cleaned_vehicles.csv",index=False)
print(f"Saved {len(df)} records to cleaned_vehicles.csv")

Saved 3555 records to cleaned_vehicles.csv
